In [1]:
import numpy as np
import pandas as pd


def calculate_array_coverage(real_start, real_end, start_indices):
    real_length = real_end - real_start + 1
    coverage_mask = [False] * real_length
    end_indices = start_indices + 39

    for start, end in zip(start_indices, end_indices):
        overlap_start = max(real_start, start)
        overlap_end = min(real_end, end)

        if overlap_start <= overlap_end:
            for i in range(overlap_start, overlap_end + 1):
                coverage_mask[i - real_start] = True

    total_overlap = sum(coverage_mask)
    coverage = total_overlap / real_length
    return coverage

file_path1 = './/results/coords/fass_ntm_domain_test.ProteinBERTcomb123456.csv'
file_path2 = './/data(test)/coords/fass_ntm_domain_test_coord.csv'

data1 = pd.read_csv(file_path1, sep="\t")
data2 = pd.read_csv(file_path2, sep="\t")
merged_data = pd.merge(data1, data2, on='id')

def convert_to_int_list(indexes_str):
    if indexes_str == '[]':
        return []
    else:
        return list(map(int, indexes_str.strip('[]').split()))

merged_data['indexes'] = merged_data['indexes'].apply(convert_to_int_list)

merged_data['coverage'] = merged_data.apply(
    lambda row: calculate_array_coverage(row['real_start'], row['real_end'], np.array(row['indexes'])), axis=1
)

def calculate_differences(row):
    if not row['indexes']:
        return np.nan, np.nan
    predicted_start = row['indexes'][0]
    predicted_end = row['indexes'][-1] + 39
    diff_start = predicted_start - row['real_start']
    diff_end = predicted_end - row['real_end']
    return diff_start, diff_end

merged_data[['diff_start', 'diff_end']] = merged_data.apply(
    lambda row: pd.Series(calculate_differences(row)), axis=1
)

merged_data

,id,cutoff,indexes,real_start,real_end,coverage,diff_start,diff_end
0,CAP61093.1,0.5,[],3,22,0.0,NaN,NaN
1,CCF38037.1,0.5,[],17,37,0.0,NaN,NaN
2,CEF83062.1,0.5,[],8,30,0.0,NaN,NaN
3,CEG02763.1,0.5,[],11,26,0.0,NaN,NaN
4,CEJ92447.1,0.5,[],8,24,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...
57,OWT42487.1,0.5,"[7, 10]",15,27,1.0,-8.0,22.0
58,PIA96287.1,0.5,[],36,53,0.0,NaN,NaN
59,POR32818.1,0.5,[],6,26,0.0,NaN,NaN
60,SCO75794.1,0.5,[],5,21,0.0,NaN,NaN
